In [112]:
from flask import Flask, render_template, request, session
import pandas as pd
from tensorflow.keras.models import load_model
import numpy as np

In [113]:
app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

In [114]:
model = load_model('model.h5')

In [115]:
def process_csv(csv_file):
    df = pd.read_csv(csv_file)
    return df

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    csv_file = request.files['csv_file']
    session['csv_data'] = pd.read_csv(csv_file).to_json()
    image_file = request.files['image_file']
    image_url = 'static/' + image_file.filename
    image_file.save(image_url)
    csv_data_html = pd.read_json(session['csv_data']).to_html(index=False)
    return render_template('index.html', csv_data=csv_data_html, image_url=image_url)

@app.route('/detect', methods=['POST'])
def detect():
    csv_data = pd.read_json(session.get('csv_data', '[]'))
    if csv_data.empty:
        return "CSV data not found"
    prediction = model.predict(np.array(csv_data))
    detection_result = "Fake Tweet" if prediction > 0.5 else "Real Tweet"
    return render_template('index.html', detection_result=detection_result)


In [116]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/May/2024 09:17:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 09:17:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/May/2024 09:17:55] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 09:17:55] "GET /static/Tweet.png HTTP/1.1" 200 -


1/1 [==============================] - 0s 170ms/step


127.0.0.1 - - [08/May/2024 09:18:15] "POST /detect HTTP/1.1" 200 -
